In [15]:
import torch
import torchvision
import sys
import numpy as np
import matplotlib.pyplot as plt
import supervision as sv
from skimage import io
from segment_anything import sam_model_registry, SamPredictor
import numpy as np
import os
import cv2
from PIL import Image
import imageio
from tensorflow.python.ops import image_ops
import pandas as pd
import time
from tqdm import tqdm

print("PyTorch version:", torch.__version__)
print("Torchvision version:", torchvision.__version__)

#Check for CUDA on Windows
print("CUDA is available:", torch.cuda.is_available())

#Check on GPU for Mac
# this ensures that the current MacOS version is at least 12.3+
print(torch.backends.mps.is_available())
# this ensures that the current current PyTorch installation was built with MPS activated.
print(torch.backends.mps.is_built())

PyTorch version: 2.0.1
Torchvision version: 0.15.2
CUDA is available: False
True
True


In [3]:
def show_mask(mask, ax, random_color=False):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
    else:
        color = np.array([30/255, 144/255, 255/255, 0.6])
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    ax.imshow(mask_image)

def show_points(coords, labels, ax, marker_size=375):
    pos_points = coords[labels==1]
    neg_points = coords[labels==0]
    ax.scatter(pos_points[:, 0], pos_points[:, 1], color='green', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)
    ax.scatter(neg_points[:, 0], neg_points[:, 1], color='red', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)

def show_box(box, ax):
    x0, y0 = box[0], box[1]
    w, h = box[2] - box[0], box[3] - box[1]
    ax.add_patch(plt.Rectangle((x0, y0), w, h, edgecolor='green', facecolor=(0,0,0,0), lw=2))

In [4]:
#Choix du device
device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')

# chargement du modèle
sam_checkpoint = 'Model/sam_vit_h/sam_vit_h_4b8939.pth'
model_type = "vit_h"

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)

predictor = SamPredictor(sam)

In [5]:
#import du dataframe train
train_df = pd.read_csv('Dataframe/train_df.csv')
print('train_df shape =', train_df.shape)

train_df shape = (13673, 2)


In [22]:
# Définition d'une liste contenant les chemins d'accès aux différentes images
MASK_DF = train_df.loc[train_df['target'] == 'BAS'][100:200]
IMAGE_PATH = np.array(MASK_DF['Path'])

# Définition des dimensions et positions des boxes et points appliqués sur chaque images pour limiter la segmentation a une partie restreinte de l'image
input_box = np.array([90, 90, 270, 270])
input_point = np.array([[180, 180]])
input_label = np.array([0])

# Création et enregistrement des masques pour chaque image
# for i in range(len(df.filepath)):
for i in tqdm(range(len(MASK_DF))):
  # lecture de l'image
  image_bgr = cv2.imread(IMAGE_PATH[i])
  # transformation en RGB
  image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)

  start_time = time.time()
  predictor.set_image(image_rgb)
  end_time = time.time()
  set_image_time = end_time - start_time
  print("Temps pris par predictor.set_image() :", set_image_time, "secondes")

  start_time = time.time()
  masks, _, _ = predictor.predict(
    point_coords=None,
    point_labels=None,
    box=input_box,
    multimask_output=False,
    )
  end_time = time.time()
  set_image_time = end_time - start_time
  print("Temps pris par predictor.predict() :", set_image_time, "secondes")

  # Chemin du dossier où vous souhaitez enregistrer l'image
  output_folder = 'Images/mask/' + IMAGE_PATH[i].split('/')[2]

  # Nom du fichier de sortie
  output_filename = str(MASK_DF['Path'].iloc[i]).split('/')[3].split('.')[0]+'_mask.png'

  # Chemin complet du fichier de sortie
  output_path = output_folder+'/'+output_filename

  mask_image = np.uint8(masks[0] * 255)

  # Save the mask image
  cv2.imwrite(output_path, mask_image)
  
  # Enregistrez l'image dans le fichier PNG
  print('L\'image n°', i, 'a été enregistrée sous :', output_path)

  0%|          | 0/100 [00:00<?, ?it/s]

Temps pris par predictor.set_image() : 28.72443413734436 secondes


  1%|          | 1/100 [00:34<56:45, 34.40s/it]

Temps pris par predictor.predict() : 5.5331621170043945 secondes
L'image n° 0 a été enregistrée sous : Images/mask/basophil/BA_731815_mask.png
Temps pris par predictor.set_image() : 20.158591985702515 secondes


  2%|▏         | 2/100 [01:00<47:57, 29.36s/it]

...

Temps pris par predictor.predict() : 7.007172107696533 secondes
L'image n° 97 a été enregistrée sous : Images/mask/basophil/BA_114542_mask.png
Temps pris par predictor.set_image() : 33.907564878463745 secondes


 99%|█████████▉| 99/100 [54:17<00:35, 35.27s/it]

Temps pris par predictor.predict() : 8.734473943710327 secondes
L'image n° 98 a été enregistrée sous : Images/mask/basophil/BA_360221_mask.png
Temps pris par predictor.set_image() : 31.788437843322754 secondes


100%|██████████| 100/100 [54:56<00:00, 32.97s/it]

Temps pris par predictor.predict() : 7.577715873718262 secondes
L'image n° 99 a été enregistrée sous : Images/mask/basophil/BA_760001_mask.png


In [14]:
BAS_DF = train_df.loc[train_df['target'] == 'BAS'][0:100]
BAS_DF

,Path,target
16,Images/PBC_dataset_normal_DIB/basophil/BA_9440...,BAS
20,Images/PBC_dataset_normal_DIB/basophil/BA_6127...,BAS
23,Images/PBC_dataset_normal_DIB/basophil/BA_8162...,BAS
53,Images/PBC_dataset_normal_DIB/basophil/BA_8334...,BAS
62,Images/PBC_dataset_normal_DIB/basophil/BA_2362...,BAS
...,...,...
1357,Images/PBC_dataset_normal_DIB/basophil/BA_1624...,BAS
1383,Images/PBC_dataset_normal_DIB/basophil/BA_6980...,BAS
1388,Images/PBC_dataset_normal_DIB/basophil/BA_7275...,BAS
1416,Images/PBC_dataset_normal_DIB/basophil/BA_4689...,BAS


In [13]:
train_df['target'].value_counts()

NEU     2644
EOS     2455
IG      2319
PLAT    1903
ERB     1252
MON     1141
BAS      991
LYM      968
Name: target, dtype: int64

In [19]:
BAS_DF = train_df.loc[train_df['target'] == 'BAS'][0:100]
BAS_DF.Path

16      Images/PBC_dataset_normal_DIB/basophil/BA_9440...
20      Images/PBC_dataset_normal_DIB/basophil/BA_6127...
23      Images/PBC_dataset_normal_DIB/basophil/BA_8162...
53      Images/PBC_dataset_normal_DIB/basophil/BA_8334...
62      Images/PBC_dataset_normal_DIB/basophil/BA_2362...
                              ...                        
1357    Images/PBC_dataset_normal_DIB/basophil/BA_1624...
1383    Images/PBC_dataset_normal_DIB/basophil/BA_6980...
1388    Images/PBC_dataset_normal_DIB/basophil/BA_7275...
1416    Images/PBC_dataset_normal_DIB/basophil/BA_4689...
1426    Images/PBC_dataset_normal_DIB/basophil/BA_5118...
Name: Path, Length: 100, dtype: object

In [20]:
train_df.head(25)

,Path,target
0,Images/PBC_dataset_normal_DIB/neutrophil/BNE_9...,NEU
1,Images/PBC_dataset_normal_DIB/neutrophil/SNE_8...,NEU
2,Images/PBC_dataset_normal_DIB/neutrophil/BNE_2...,NEU
3,Images/PBC_dataset_normal_DIB/neutrophil/SNE_8...,NEU
4,Images/PBC_dataset_normal_DIB/erythroblast/ERB...,ERB
5,Images/PBC_dataset_normal_DIB/eosinophil/EO_36...,EOS
6,Images/PBC_dataset_normal_DIB/neutrophil/SNE_9...,NEU
7,Images/PBC_dataset_normal_DIB/eosinophil/EO_16...,EOS
8,Images/PBC_dataset_normal_DIB/lymphocyte/LY_10...,LYM
9,Images/PBC_dataset_normal_DIB/eosinophil/EO_31...,EOS


In [2]:
# essai ludo VSCode pull request
print("essai ludo VSCode pull request")

essai ludo VSCode pull request
